<a href="https://colab.research.google.com/github/silaspc82/FIAP_IA_PARA_DEVS_TECH_CHALLENGE/blob/main/Tech_Challenge_Fase_03/FIAP_TechChallengeFase03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Descompactando os arquivo do modelo Llama 1.3M

In [ ]:
!unzip -q "/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M.raw.zip" -d "/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/"

Descompactando o arquivo json de treino do modelo

In [ ]:
!gunzip -k "/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M/trn.json.gz"

Este código realiza a leitura e limpeza de dados do arquivo JSON "trn.json" contendo informações sobre produtos (títulos e descrições) e os armazena em uma lista após a aplicação de algumas transformações.

Bibliotecas utilizadas:
  * json: Usada para carregar e manipular dados em formato JSON.
  * html: Utilizada para converter entidades HTML (como &#8217;) para seus equivalentes em texto.
  * re: A biblioteca de expressões regulares, usada para encontrar e substituir padrões em textos.

Preparando os dados de produtos para o treino:
  * Carrega um arquivo JSON contendo produtos com seus títulos e descrições.
  * Converte entidades HTML para caracteres normais.
  * Remove caracteres especiais indesejados dos títulos e descrições, mantendo apenas letras, números, vírgulas e pontos.
  * Filtra dados inválidos (linhas com título ou descrição vazios).
  * Armazena os dados limpos em uma lista de produtos para uso posterior no treinamento.


In [ ]:
import json
import html
import re

def converter_entidades_html(texto):
    return html.unescape(texto)

def limpar_caracteres_especiais(texto):
    # Definir uma expressão regular para manter apenas letras, números e espaços
    texto_limpo = re.sub(r'[^A-Za-z0-9 ,.]', '', texto)
    return texto_limpo

with open("/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M/trn.json", "r") as arquivo:

  lista_produtos = []
  contador_linhas = 0
  for linha in arquivo:
    contador_linhas += 1

    # linha = '{"uid": "0001360000", "title": "Mog\'s Kittens", "content": "Judith Kerr&#8217;s best&#8211;selling adventures of that endearing (and exasperating) cat Mog have entertained children for more than 30 years. Now, even infants and toddlers can enjoy meeting this loveable feline. These sturdy little board books&#8212;with their bright, simple pictures, easy text, and hand&#8211;friendly formats&#8212;are just the thing to delight the very young. Ages 6 months&#8211;2 years.", "target_ind": [146, 147, 148, 149, 495], "target_rel": [1.0, 1.0, 1.0, 1.0, 1.0]}'
    json_linha = json.loads(linha)
    # print(json_linha)

    pular = False

    title = json_linha['title']
    title = converter_entidades_html(title)
    title = limpar_caracteres_especiais(title)
    title = title.strip()


    content = json_linha['content']
    content = converter_entidades_html(content)
    content = limpar_caracteres_especiais(content)
    content = content.strip()


    # print(f"Title: {title}")
    # print(f"Content: {content}")
    # break

    if not title:
      pular = True

    if not content:
      pular = True

    if pular:
      continue

    lista_produtos.append({
        "title": title,
        "content": content,
    })

  print(f"Total de linhas do arquivo original: {contador_linhas}")
  print(f"Total de linhas após a limpesa e tratado dos dados: {len(lista_produtos)}")


Total de linhas do arquivo original: 2248619
Total de linhas após a limpesa e tratado dos dados: 1389837


In [ ]:
CAMINHHO_CSV = "/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M/"

Excluindo os registros duplicados no arquivo de produtos e gerando um novo arquivo CSV com os dados tratados

In [ ]:
import pandas as pd

df = pd.DataFrame(lista_produtos)

df_novo = df.drop_duplicates(subset=['title', 'content'], keep='first')
df_novo.reset_index(drop=True)
df_novo = df_novo.dropna()
df_novo['title'] = df_novo['title'].astype('str')
df_novo['content'] = df_novo['content'].astype('str')

df_novo.to_csv(f'{CAMINHHO_CSV}trn_tratado.csv', index=False, encoding='utf-8')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1389837 entries, 0 to 1389836
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   title    1389837 non-null  object
 1   content  1389837 non-null  object
dtypes: object(2)
memory usage: 21.2+ MB


In [ ]:
df_novo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1366488 entries, 0 to 1389836
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   title    1366488 non-null  object
 1   content  1366488 non-null  object
dtypes: object(2)
memory usage: 31.3+ MB


Avaliando a estrutura dos dados resultantes

In [ ]:
df_novo.head()

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
1,Mogs Kittens,Judith Kerrs bestselling adventures of that en...
2,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 28 years. Perfect fo...
3,The Prophet,"In a distant, timeless place, a mysterious pro..."
4,Rightly Dividing the Word,This text refers to thePaperbackedition.


Verificando que não existem dados nulos ou duplicados

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1389837 entries, 0 to 1389836
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   title    1389837 non-null  object
 1   content  1389837 non-null  object
dtypes: object(2)
memory usage: 21.2+ MB


Gerando arquivo CSV com os dados de produtos tratados

In [ ]:
import pandas as pd

df = pd.read_csv(f'{CAMINHHO_CSV}trn_tratado.csv', encoding='utf-8')
df['title'] = df['title'].astype('string')
df['content'] = df['content'].astype('string')
df = df.dropna()
df.head()

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
1,Mogs Kittens,Judith Kerrs bestselling adventures of that en...
2,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 28 years. Perfect fo...
3,The Prophet,"In a distant, timeless place, a mysterious pro..."
4,Rightly Dividing the Word,This text refers to thePaperbackedition.


Dividindo o arquivo de 100000 em 100000 mil linhas para agilizar e reduzir o tempo de processamento

In [ ]:
CAMINHO_ARQ_PARTICIONADOS = f'{CAMINHHO_CSV}/Arquivos_Particionados'

In [ ]:
# Função para dividir o DataFrame em arquivos com n linhas
def dividir_dataframe_em_arquivos(df, linhas_por_arquivo):
    # Número total de arquivos
    num_arquivos = len(df) // linhas_por_arquivo + (1 if len(df) % linhas_por_arquivo != 0 else 0)
    print(f'Numero de arquivos: {num_arquivos}')

    for i in range(num_arquivos):
        # Pega o pedaço correspondente
        df_chunk = df[i * linhas_por_arquivo:(i + 1) * linhas_por_arquivo]

        # Salva em um arquivo CSV com o índice i no nome do arquivo
        df_chunk.to_csv(f'{CAMINHO_ARQ_PARTICIONADOS}/trn_tratado_{i + 1}.csv', index=False)

# Dividir o DataFrame em arquivos com 100.000 linhas cada
dividir_dataframe_em_arquivos(df, 100000)

Numero de arquivos: 14


Abrindo o primeiro arquivo particionado

In [ ]:
import pandas as pd

CAMINHO_TRATAMENTO = f'{CAMINHO_ARQ_PARTICIONADOS}/trn_tratado_1'
df = pd.read_csv(f'{CAMINHO_TRATAMENTO}.csv', encoding='utf-8')
df = df.dropna()
df['title'] = df['title'].astype('string')
df['content'] = df['content'].astype('string')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   title    100000 non-null  string
 1   content  100000 non-null  string
dtypes: string(2)
memory usage: 1.5 MB


In [ ]:
df.head()

,title,content
0,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...
1,Mogs Kittens,Judith Kerrs bestselling adventures of that en...
2,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 28 years. Perfect fo...
3,The Prophet,"In a distant, timeless place, a mysterious pro..."
4,Rightly Dividing the Word,This text refers to thePaperbackedition.


Instalação da biblioteca unsloth

A biblioteca Unsloth foi criada para facilitar e acelerar o processo de fine-tuning de grandes modelos de linguagem, como Llama-3, Mistral, e outros. Ela oferece um desempenho mais rápido, com menor uso de memória, permitindo uma otimização eficiente sem perda de precisão.

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
#!pip show pandas pyarrow
#!pip uninstall pandas pyarrow
!pip install pyarrow==15.0.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 24.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 17.0.0
    Uninstalling pyarrow-17.0.0:
      Successfully uninstalled pyarrow-17.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 15.0.0 which is incompatible.


In [ ]:
#!pip install --upgrade pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 104.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.1.4
    Uninstalling pandas-2.1.4:
      Successfully uninstalled pandas-2.1.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pandas<2.2.2dev0,>=2.0, but you have pandas 2.2.3 which is incompatible.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
google-colab 1.0.0 requires pandas==2.1.4, but you have pandas 2.2.3 which is incompatible.


In [ ]:
#!pip install pandas==2.1.4 pyarrow==14.0.1

Liberando a memória

In [ ]:
import torch
import gc

torch.cuda.empty_cache()
gc.collect()

106

Usando o módulo ***FastLanguageModel*** da biblioteca ***Unsloth*** para carregar o modelo base do Hugging Face e setando parâmetros iniciais.

A biblioteca ***FastLanguageModel*** é um módulo específico da Unsloth, projetada para otimizar o treinamento e a inferência de grandes modelos de linguagem. Ela facilita o fine-tuning de modelos como Llama e Mistral ao oferecer suporte para quantização de parâmetros e economia de memória, o que torna o processo mais rápido e eficiente.

Descrição dos parâmetros do FastLanguageModel:
  * model_name: Nome do modelo pré-treinado a ser carregado (ex: “unsloth/Meta-Llama-3.1-8B-bnb-4bit”).
  * max_seq_length: Comprimento máximo de tokens que o modelo pode processar (ex: 2048).
  * load_in_4bit: Carrega o modelo quantizado em 4 bits, economizando memória e acelerando o processamento.
  * lora_alpha: Controla a regularização do ajuste LoRA (Low-Rank Adaptation).
  * lora_dropout: Taxa de dropout aplicada nas camadas LoRA, para evitar overfitting.
  * target_modules: Define quais partes do modelo serão otimizadas com LoRA (ex: “q_proj”, “v_proj”).
  * use_gradient_checkpointing: Ativa o checkpointing de gradiente para economizar memória durante o treinamento.
  * dtype: Define o tipo de dado a ser usado nas operações do modelo (ex: float16, bfloat16).

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/tinyllama-chat-bnb-4bit",
    # model_name = "unsloth/SmolLM-360M",
    # model_name = "unsloth/SmolLM-360M-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

Aplicando o método ***LoRA*** (Low-Rank Adaptation) e ***PEFT*** (Parameter-Efficient Fine-Tuning), que permitem treinar modelos grandes com um número muito menor de parâmetros atualizados, acelerando o treinamento e diminuindo os requisitos de memória.

Descrição dos parâmetros do LoRA:
  * model,                # O modelo base, ex: Llama
  * r=16,                 # Rank para LoRA (controla a dimensionalidade)
  * lora_alpha=16,        # Hiperparâmetro de regularização
  * lora_dropout=0,       # Taxa de dropout nas camadas LoRA
  * target_modules=[      # Partes do modelo que serão ajustadas "q_proj", "k_proj", "v_proj"]...
  * use_rslora=True,      # Usa a técnica RSLoRA para mais eficiência
  * use_gradient_checkpointing="unsloth"  # Economiza memória durante o treinamento


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.1, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.9.post4 patched 22 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Após aplicar o PEFT e LoRA, preparamos o processamento de prompts para treinar um modelo de linguagem com base em um dataset de produtos.

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["title"]
    outputs      = examples["content"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

def add_instruction_column(example):
    example["instruction"] =  f"Describe the product"
    return example


from datasets import load_dataset, Dataset
# dataset = load_dataset("LuizfvFonseca/trn_limpo_parte_1_de_70", split = "train")
# dataset = load_dataset("csv", data_files=f'{CAMINHO_TRATAMENTO}.csv', split = "train")
dataset = Dataset.from_pandas(df, split = "train")
# Implementar o carregamento do dataset de um diretório local contendo o dataset do hugginface.


# Aplica a função add_instruction_column ao dataset
dataset = dataset.map(add_instruction_column)
dataset = dataset.map(formatting_prompts_func, batched = True,)

#após adicionar as colunas => title, content, instruction, text(instruction, title, content)

alpaca_prompt_text = dataset['text'][9]
print(alpaca_prompt_text)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product

### Input:
Nice for Mice

### Response:
Jill Barklem was born in Epping in 1951. After an accident when she was thirteen, Jill was unable to take part in PE or games at school and instead developed her talent for drawing and art. On leaving school, she studied illustration at St Martins in London. Jill is now a fulltime illustrator, working on the series of Brambly Hedge books.</s>



Verificando as informações do Dataset após a adição dos campos


In [ ]:
print(dataset[9])

{'title': 'Nice for Mice', 'content': 'Jill Barklem was born in Epping in 1951. After an accident when she was thirteen, Jill was unable to take part in PE or games at school and instead developed her talent for drawing and art. On leaving school, she studied illustration at St Martins in London. Jill is now a fulltime illustrator, working on the series of Brambly Hedge books.', 'instruction': 'Describe the product', 'text': 'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nDescribe the product\n\n### Input:\nNice for Mice\n\n### Response:\nJill Barklem was born in Epping in 1951. After an accident when she was thirteen, Jill was unable to take part in PE or games at school and instead developed her talent for drawing and art. On leaving school, she studied illustration at St Martins in London. Jill is now a fulltime illustrator, working on the series of Bra

Realizando inferência antes do treinamento do modelo

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Describe the product", # instruction
        "Nice for Mice", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

# outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
# tokenizer.batch_decode(outputs)

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product

### Input:
Nice for Mice

### Response:
The product is a mouse that is designed to be easy to use and maintain. It has a sleek and modern design, making it a stylish addition to any home or office. The mouse is equipped with a high-quality sensor that detects the position of the mouse, allowing for precise tracking and navigation. The mouse also features a comfortable and ergonomic design, making it comfortable to use for long periods of time. Overall, the product is designed to provide a high-quality and user-friendly experience for its users.</s>


Carregando o Dataset e o Tokenizador com as configurações para o treinamento do modelo

Resumo do Fluxo:
1. Carrega-se um dataset.
2.	Adiciona-se uma instrução fixa a cada exemplo (descrição de produto).
3.	Gera-se um prompt formatado com a instrução, o título do produto e a descrição do produto.
4.	O modelo, quando treinado, vai usar esse prompt para aprender a prever a descrição do produto baseado no título e na instrução fornecida.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 22, #testando com 16 não ficou bom
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 120, #testando com 50, 60
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        max_grad_norm = 0.6,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        torch_empty_cache_steps=15,
        group_by_length=True,
        # gradient_checkpointing=True,
        # dataloader_prefetch_factor=4,
        #auto_find_batch_size=True,
    ),
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-SXM4-40GB. Max memory = 39.564 GB.
0.865 GB of memory reserved.


Treinando o modelo com o dados do Dataset

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 22
\        /    Total batch size = 44 | Total steps = 2,272
 "-____-"     Number of trainable parameters = 12,615,680


Step,Training Loss
1,5.416000
2,5.925000
3,5.089200
4,4.792500
5,4.428800
6,4.436600
7,4.284300
8,3.469800
9,3.438300
10,3.135300


In [ ]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

13552.5521 seconds used for training.
225.88 minutes used for training.
Peak reserved memory = 11.666 GB.
Peak reserved memory for training = 10.801 GB.
Peak reserved memory % of max memory = 29.486 %.
Peak reserved memory for training % of max memory = 27.3 %.


In [ ]:
import torch
import gc

# Liberando a memória
torch.cuda.empty_cache()
gc.collect()

160

Realizando inferência após o treinamento do modelo

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Describe the product", # instruction
        "The Prophet", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 256, temperature=1, top_p=0.9, repetition_penalty=1.2)

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product

### Input:
The Prophet

### Response:
A powerful and moving novel about the life of Muhammad, from his childhood in Mecca to his final years as prophet and leader of the Muslims.</s>


Salvando o modelo no Google Drive.

In [ ]:
model.save_pretrained(f'{CAMINHO_TRATAMENTO}_lora_model') # Local saving
tokenizer.save_pretrained(f'{CAMINHO_TRATAMENTO}_lora_model')

('/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M//Arquivos_Particionados/trn_tratado_1_lora_model/tokenizer_config.json',
 '/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M//Arquivos_Particionados/trn_tratado_1_lora_model/special_tokens_map.json',
 '/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M//Arquivos_Particionados/trn_tratado_1_lora_model/tokenizer.model',
 '/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M//Arquivos_Particionados/trn_tratado_1_lora_model/added_tokens.json',
 '/content/drive/MyDrive/TI/Cursos/FIAP/IAParaDEVS/TechChallenge/Fase_03/BaseDeDados/LF-Amazon-1.3M//Arquivos_Particionados/trn_tratado_1_lora_model/tokenizer.json')

Salvando o modelo no HugginFace

In [ ]:
from google.colab import userdata

model.push_to_hub("silaspc/tinyllama-chat-bnb-4bit-g12-parte-1", token = userdata.get('HF_TOKEN')) # Online saving
tokenizer.push_to_hub("silaspc/tinyllama-chat-bnb-4bit-g12-parte-1", token = userdata.get('HF_TOKEN')) # Online saving

README.md:   0%|          | 0.00/582 [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/50.5M [00:00<?, ?B/s]

Saved model to https://huggingface.co/silaspc/tinyllama-chat-bnb-4bit-g12-parte-1


  0%|          | 0/1 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

### Carregando o modelo direto do HugginFace para testes

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        # model_name = "unsloth/tinyllama-chat-bnb-4bit",
        model_name = "silaspc/tinyllama-chat-bnb-4bit-g12-parte-1",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference


==((====))==  Unsloth 2024.9.post4: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-SXM4-40GB. Max memory: 39.564 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.1+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


adapter_model.safetensors:   0%|          | 0.00/50.5M [00:00<?, ?B/s]

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
    alpaca_prompt.format(
        "Describe the product", # instruction
        "The Prophet", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                          temperature=0.7,
                          top_p=0.9,
                          repetition_penalty=1.1
                   )

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Describe the product

### Input:
The Prophet

### Response:
Amazon Best Books of the Month, April 2013 The Prophet is a novel about the power of faith and the importance of following your heart. In this case, the heart belongs to a young woman named Aisha, who has been raised in a Muslim family but has never felt at home there. When she meets a man named Abu Bakr, she finds herself drawn to him, despite his unorthodox beliefs. As their relationship deepens, Aishas faith begins to shift, and she realizes that her life will never be the same. This is a beautifully written story about
